<a href="https://colab.research.google.com/github/mohitsharmas97/pytorch_notebooks/blob/main/Pytorch_NN_module_day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class Model(nn.Module):

    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(num_features,10),
            nn.ReLU(),
            nn.Linear(10,1),
            nn.Sigmoid()
        )

    def forward(self,features):
      out=self.network(features)

      return out
      #created model class

In [ ]:
features=torch.rand(10,5) #create a dataset

model=Model(features.shape[1]) #create a model

model(features) #call model for forward pass



tensor([[0.5370],
        [0.5317],
        [0.5231],
        [0.5379],
        [0.5391],
        [0.5414],
        [0.5370],
        [0.5361],
        [0.5263],
        [0.5339]], grad_fn=<SigmoidBackward0>)

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model,input_data=torch.rand(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 10]                  60
│    └─ReLU: 2-2                         [10, 10]                  --
│    └─Linear: 2-3                       [10, 1]                   11
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 71
Trainable params: 71
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

Dataset--breast cancer


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.shape

(569, 33)

In [ ]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [ ]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
df.shape

(569, 31)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['diagnosis']),df['diagnosis'],test_size=0.2)

In [ ]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
x_train

array([[ 1.10729985,  0.27569772,  1.07399175, ...,  1.23563867,
         0.22077428,  0.03243099],
       [ 1.17372991, -0.17408134,  1.17888735, ...,  0.24018676,
         0.15992625, -0.24796483],
       [ 2.63230288,  0.59237889,  2.51315945, ...,  1.80622128,
         1.04612319,  0.40411848],
       ...,
       [ 0.4343345 ,  0.85857466,  0.75510911, ...,  1.98414489,
         0.55933895,  0.94915144],
       [-0.26462522,  0.15407379, -0.22377668, ...,  0.28926913,
        -0.11466999,  2.19897777],
       [-0.54478676,  2.41214825, -0.62028206, ..., -1.79213032,
        -0.79491975, -0.92395785]])

In [ ]:
y_train

,diagnosis
408,M
53,M
164,M
231,B
300,M
...,...
174,B
210,M
258,M
465,B


In [ ]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)


In [ ]:
y_train

array([1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,

In [ ]:
x_train_tensor=torch.from_numpy(x_train).float() # Convert to float
x_test_tensor=torch.from_numpy(x_test).float() # Convert to float
y_train_tensor=torch.from_numpy(y_train).float() # Convert to float
y_test_tensor=torch.from_numpy(y_test).float() # Convert to float

In [ ]:
x_train_tensor.shape

torch.Size([455, 30])

In [ ]:
y_train_tensor.shape

torch.Size([455])

In [ ]:
#commented part Full-Batch Gradient Descent (not prefered more only for understanding )


In [ ]:
# class MySimpleNN(nn.Module):
#   def __init__(self,input_features):
#     super().__init__() ##n module ko srtart
#     self.linear=nn.Linear(input_features,1) #input_features=30   and 1 is o/p
#     self.sigmoid=nn.Sigmoid()

#   def forward(self,features):
#     out=self.linear(features)
#     out=self.sigmoid(out)
#     return out



In [ ]:
# learning_rate = 0.1
# epochs = 25

In [ ]:
# loss_function=nn.BCELoss() #built in loss function

In [ ]:
# type(loss_function)


In [ ]:
# model=MySimpleNN(x_train_tensor.shape[1])

# optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)


# for epoch in range(epochs):

#   #forward pass
#   y_pred=model(x_train_tensor)

#   loss=loss_function(y_pred,y_train_tensor.reshape(-1, 1)) # Call loss_function directly and reshape y_train_tensor


# #clear gradients
#   optimizer.zero_grad()

# #backward pass
#   loss.backward()

# #parameters update
#   optimizer.step()




# print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

In [ ]:
# # model evaluation
# with torch.no_grad():
#   y_pred = model.forward(x_test_tensor) # Corrected variable name
#   y_pred = (y_pred > 0.5).float() # Changed threshold to 0.5 for binary classification
#   accuracy = (y_pred == y_test_tensor).float().mean()
#   print(f'Accuracy: {accuracy.item()}')

In [ ]:
#now we apply mini batch gradient descent with the help dataset and dataloader class--Mini-Batch Gradient Descent

In [ ]:
from torch.utils.data import Dataset,DataLoader

In [ ]:
from torch.utils.data import TensorDataset

class CustomDataset(Dataset):
    def __init__(self, features,labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
train_dataset=CustomDataset(x_train_tensor,y_train_tensor)
test_dataset=CustomDataset(x_test_tensor,y_test_tensor)

NameError: name 'CustomDataset' is not defined

In [ ]:
train_dataset[10]

(tensor([-1.2446, -0.8671, -1.2253, -1.0408, -0.1454, -0.4215, -0.8528, -0.6764,
         -1.5655,  0.8417, -0.5683, -0.0322, -0.4468, -0.5417,  0.5396, -0.3133,
         -0.7043, -0.8437, -0.2993, -0.1347, -1.0694, -0.4723, -1.0097, -0.8989,
          0.3920, -0.2216, -0.9286, -0.7908, -0.6498,  0.4161]),
 tensor(0.))

In [ ]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=True)

In [ ]:
#defining model


import torch.nn as nn


class MySimpleNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.linear = nn.Linear(input_features, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, features):
          out = self.linear(features)
          out = self.sigmoid(out)
          return out



In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
#create model
model=MySimpleNN(x_train_tensor.shape[1])

#optimizer
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

#define loss function
loss_function=nn.BCELoss()

#training pipeline

In [ ]:
for epoch in range(epochs):
  total_loss = 0
  for batch_features,batch_labels in train_loader:

    #forward pass
    y_pred=model(batch_features)

    #calculate loss
    loss=loss_function(y_pred,batch_labels.reshape(-1,1))
    total_loss += loss.item()

    #clear gradients
    optimizer.zero_grad()

    #backward pass
    loss.backward()

    #update parameters
    optimizer.step()

  print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.06645411252975464
Epoch 2, Loss: 0.010042754001915455
Epoch 3, Loss: 0.03840559720993042
Epoch 4, Loss: 0.060824207961559296
Epoch 5, Loss: 0.005916473921388388
Epoch 6, Loss: 0.08587885648012161
Epoch 7, Loss: 0.005271356087177992
Epoch 8, Loss: 0.09587346762418747
Epoch 9, Loss: 0.005204121582210064
Epoch 10, Loss: 0.009341160766780376
Epoch 11, Loss: 0.11188596487045288
Epoch 12, Loss: 0.008886375464498997
Epoch 13, Loss: 0.001742495340295136
Epoch 14, Loss: 0.03863086178898811
Epoch 15, Loss: 0.037886254489421844
Epoch 16, Loss: 0.07790511101484299
Epoch 17, Loss: 0.12037694454193115
Epoch 18, Loss: 0.07749151438474655
Epoch 19, Loss: 0.0276749636977911
Epoch 20, Loss: 0.003541966201737523
Epoch 21, Loss: 0.0037520970217883587
Epoch 22, Loss: 0.002836128929629922
Epoch 23, Loss: 0.03867786005139351
Epoch 24, Loss: 0.01885669119656086
Epoch 25, Loss: 0.11710920184850693


In [ ]:
model.eval()
accuracy_list=[]

with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    y_pred=model(batch_features)
    y_pred=(y_pred>0.8).float()

    batch_accuracy = (y_pred == batch_labels.reshape(-1, 1)).float().mean()
    accuracy_list.append(batch_accuracy.item())

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')

Accuracy: 0.9766
